# Load the necessary modules

Zipline only works with an older version of pandas. It is necessary to downgrade to the version of pandas that is 0.22.0

In [22]:
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import zipline
from zipline.api import order, record, symbol, set_benchmark
from zipline import run_algorithm
from collections import OrderedDict
from datetime import datetime

# Load and Preprocess the data with zipline

In [55]:
# Store the different datasets
data = OrderedDict()
# Loads the data from the csv file
stocks  = ["AAPL"]

# Iterate to load stock data from csv
for stock in stocks:
    data[stock] = pd.read_csv("{}.csv".format(stock), index_col =0, parse_dates=['Date'])
    data[stock] = data[stock][["open","high","low","close","volume"]]
    data[stock] = data[stock].resample('1d').mean()
    data[stock].fillna(method='ffill', inplace=True)
    print(data[stock].head(10))

             open   high    low  close      volume
Date                                              
2013-01-02  38.54  38.99  38.48  38.85  15826400.0
2013-01-03  39.28  39.52  38.71  38.88  14937200.0
2013-01-04  38.90  39.30  38.69  39.19  10376000.0
2013-01-05  38.90  39.30  38.69  39.19  10376000.0
2013-01-06  38.90  39.30  38.69  39.19  10376000.0
2013-01-07  39.24  39.69  39.20  39.47  10242400.0
2013-01-08  39.41  39.84  39.25  39.84  11800400.0
2013-01-09  40.35  40.64  39.93  40.45  17552800.0
2013-01-10  40.51  40.69  39.82  40.13  16313600.0
2013-01-11  39.94  40.31  39.88  40.29   9578000.0


In [56]:
# Convert data to panel
StockPrice = pd.Panel(data)
StockPrice.minor_axis = ["open","high","low","close","volume"]
StockPrice.major_axis = StockPrice.major_axis.tz_localize(pytz.utc)
print(StockPrice)

<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 2187 (major_axis) x 5 (minor_axis)
Items axis: AAPL to AAPL
Major_axis axis: 2013-01-02 00:00:00+00:00 to 2018-12-28 00:00:00+00:00
Minor_axis axis: open to volume


In [36]:
%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [57]:
# Define necessary two functions to run algorithm
def initialize(context):
    set_benchmark(symbol('AAPL'))
    
def handle_data(context, data):
    order(symbol('AAPL'),10)
    record(AAPL = data.current(symbol('AAPL'),'price'))
    

In [58]:
perf = zipline.run_algorithm(start=datetime(2013,1,2,0,0,0,0,pytz.utc),
                            end=datetime(2018,12,28,0,0,0,0,pytz.utc),
                            initialize=initialize,
                            data_frequency='daily',
                            capital_base=10000,
                            handle_data=handle_data,
                            data=StockPrice)
perf.head(5)

,AAPL,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2013-01-02 21:00:00+00:00,38.85,NaN,0.000000,NaN,0.008044,NaN,NaN,0.00000,10000.00000,0.0,...,0.0,0.0,0,NaN,10000.00000,0.0,0.0,1,[],0.0
2013-01-03 21:00:00+00:00,38.88,0.000229,-0.000020,-0.005698,0.008822,0.081621,0.002811,-389.00440,9610.99560,388.8,...,0.0,0.0,0,-11.224972,10000.00000,0.0,0.0,2,"[{'amount': 10, 'dt': 2013-01-03 21:00:00+00:0...",0.0
2013-01-04 21:00:00+00:00,39.19,0.002751,0.000269,-0.009178,0.016866,0.066323,0.022529,-392.10595,9218.88965,783.8,...,0.0,0.0,0,120.604656,9610.99560,388.8,388.8,3,"[{'amount': 10, 'dt': 2013-01-04 21:00:00+00:0...",0.0
2013-01-07 21:00:00+00:00,39.47,0.004216,0.000808,-0.002835,0.024131,0.055530,0.035644,-394.90735,8823.98230,1184.1,...,0.0,0.0,0,313.797591,9218.88965,783.8,783.8,4,"[{'amount': 10, 'dt': 2013-01-07 21:00:00+00:0...",0.0
2013-01-08 21:00:00+00:00,39.84,0.007274,0.001897,-0.040178,0.033731,0.053779,0.080866,-398.60920,8425.37310,1593.6,...,0.0,0.0,0,658.626880,8823.98230,1184.1,1184.1,5,"[{'amount': 10, 'dt': 2013-01-08 21:00:00+00:0...",0.0
